In [1]:
import pandas as pd
import lightgbm as lgb
import os
import gc
import pickle
from sklearn import preprocessing, model_selection

In [2]:
#"""
def get_type_feature_all(sample,train_df,key,on,type_c,mark):
    pickle_path="../pickle/"
    filename = "_".join([mark+"_%s_features"%type_c, "_".join(key), on, str(len(sample))]) + ".pkl"
    try:
        with open(pickle_path + filename, "rb") as fp:
            print("load {} {} feature from pickle file: key: {}, on: {}...".format(mark,type_c,"_".join(key), on))
            col = pickle.load(fp)
        for c in col.columns:
            sample[c] = col[c]
        gc.collect()
    except:
        print('get {} {} feature, key: {}, on: {}'.format(mark,type_c,"_".join(key), on))
        if type_c=="count":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].count()).reset_index()
        if type_c=="mean":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].mean()).reset_index()
        if type_c=="nunique":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].nunique()).reset_index()
        if type_c=="max":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].max()).reset_index()
        if type_c=="min":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].min()).reset_index()
        if type_c=="sum":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].sum()).reset_index()
        if type_c=="std":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].std()).reset_index()
        if type_c=="median":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].median()).reset_index()
        tmp.columns = key+[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
        tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on] = tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on].astype('float32')
        sample = sample.merge(tmp, on=key, how='left')
        with open(pickle_path + filename, "wb") as fp:
            col = sample[[mark+"_"+"_".join(key) + '_%s_'%type_c + on]].copy()
            pickle.dump(col, fp)
        del tmp
    #if save_only:
        #for i in col.columns:
            #del sample[i]
    del col,train_df,sample[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on


In [3]:
train_data=pd.read_csv("../data/train.csv")
test_data=pd.read_csv("../data/test.csv")
sub=pd.read_csv("../data/sample_submission.csv")

In [43]:
def create_features(data):
    data["order_1h_2h"]=data["orders_1h"]+data["orders_2h"]
    
    date_max=data.groupby(["date"]).article_id.max().reset_index()
    date_max.columns=["date","date_max"]
    data_min=date_max.copy()
    data_min.columns=["date","date_min"]
    data_min["date"]=data_min["date"]+1
    
    data=data.merge(date_max,on="date",how="left")
    data=data.merge(data_min,on="date",how="left")
    data["date_min"]=data["date_min"].fillna(0)
    
    data["article_id"]=(data["article_id"]-data["date_min"])/(data["date_max"]-data["date_min"])
    
    data['hour']=data["article_id"].apply(lambda x:int(x*24))
    
    data["dayofweek"]=data["date"]%7
    
    del data["date_min"],data["date_max"]
    
    data["comments_1h_2h"]=data["comments_1h"]+data["comments_2h"]
    data["zhi_1h_2h"]=data["zhi_1h"]+data["zhi_2h"]
    data["buzhi_1h_2h"]=data["buzhi_1h"]+data["buzhi_2h"]
    data["favorite_1h_2h"]=data["favorite_1h"]+data["favorite_2h"]

    
    cat_features=[['baike_id_1h'],['author'],['level1'], ['level2'], ['level3'], ['level4'], ['brand'], ['mall'], ['url'],['dayofweek'],
                  ['author','level1'],['author','level2'],['author','level3'],['author','level4'],['author','brand'],['author','mall'],
                  ['mall','brand'],['mall','level1'],['mall','level2'],['mall','level3'],['mall','level4'],
                  ['brand','level1'],['brand','level2'],['brand','level3'],['brand','level4'],
                  ['baike_id_1h','mall'],
                  ["mall","brand","level1"],["mall","brand","level2"],["mall","brand","level3"],["mall","brand","level4"]


                 ]
    num_features=[#'comments_1h_2h','zhi_1h_2h','buzhi_1h_2h','favorite_1h_2h',
                  #'comments_1h','comments_2h',
                  #'zhi_1h','zhi_2h',
                  #'buzhi_1h','buzhi_2h',
                  #'favorite_1h','favorite_2h',
                  'orders_1h','orders_2h','order_1h_2h','article_id'
    ]
    #"""
    for cat in cat_features:
        #data,_=get_type_feature_all(data, data, cat+["hour"],"article_id" , "mean", "fe")
        #data,_=get_type_feature_all(data, data, cat,"article_id" , "count", "fe")
        
        #data,_=get_type_feature_all(data, data, cat,"price" , "mean", "fe")
        #data,_=get_type_feature_all(data, data, cat,"price" , "max", "fe")
        #data,_=get_type_feature_all(data, data, cat,"price" , "min", "fe")
        
        for num in num_features:
            data,_=get_type_feature_all(data, data, cat,num , "mean", "fe")
            data,_=get_type_feature_all(data, data, cat,num , "sum", "fe")
    #"""
    return data

In [44]:
data=pd.concat([train_data,
                #test_data,
               ]).reset_index(drop=True)
data=create_features(data)

get fe mean feature, key: baike_id_1h, on: comments_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: comments_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: zhi_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: zhi_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: buzhi_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: buzhi_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: favorite_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: baike_id_1h, on: orders_1h...
load fe sum feature from pickle file: key: baike_id_1h, on: orders_1h...
load fe mean feature from pickle file: key: baike_id_1h, on: orders_2h...
load fe sum feature from pickle file: key: baike_id_1h, on: orders_2h...
load fe mean feature from pickle file: key: baike_id_1h, on: order_1h_2h...
load fe sum feature from pickle file: key: baike_id_1h, on: order_1h_2h...
load fe mean feature from pickle file: key: baike_id_1h, on: article_id...
load fe sum 

get fe sum feature, key: url, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: url, on: orders_1h...
load fe sum feature from pickle file: key: url, on: orders_1h...
load fe mean feature from pickle file: key: url, on: orders_2h...
load fe sum feature from pickle file: key: url, on: orders_2h...
load fe mean feature from pickle file: key: url, on: order_1h_2h...
load fe sum feature from pickle file: key: url, on: order_1h_2h...
load fe mean feature from pickle file: key: url, on: article_id...
load fe sum feature from pickle file: key: url, on: article_id...
get fe mean feature, key: dayofweek, on: comments_1h_2h_rt
get fe sum feature, key: dayofweek, on: comments_1h_2h_rt
get fe mean feature, key: dayofweek, on: zhi_1h_2h_rt
get fe sum feature, key: dayofweek, on: zhi_1h_2h_rt
get fe mean feature, key: dayofweek, on: buzhi_1h_2h_rt
get fe sum feature, key: dayofweek, on: buzhi_1h_2h_rt
get fe mean feature, key: dayofweek, on: favorite_1h_2h_rt
get fe sum feature, key:

get fe mean feature, key: mall_brand, on: zhi_1h_2h_rt
get fe sum feature, key: mall_brand, on: zhi_1h_2h_rt
get fe mean feature, key: mall_brand, on: buzhi_1h_2h_rt
get fe sum feature, key: mall_brand, on: buzhi_1h_2h_rt
get fe mean feature, key: mall_brand, on: favorite_1h_2h_rt
get fe sum feature, key: mall_brand, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: mall_brand, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand, on: article_id...
load fe sum feature from pickle file: key: mall_brand, on: article_id...
get fe mean feature, key: mall_level1, on: comments_1h_2h_rt
get fe sum f

load fe mean feature from pickle file: key: brand_level3, on: article_id...
load fe sum feature from pickle file: key: brand_level3, on: article_id...
get fe mean feature, key: brand_level4, on: comments_1h_2h_rt
get fe sum feature, key: brand_level4, on: comments_1h_2h_rt
get fe mean feature, key: brand_level4, on: zhi_1h_2h_rt
get fe sum feature, key: brand_level4, on: zhi_1h_2h_rt
get fe mean feature, key: brand_level4, on: buzhi_1h_2h_rt
get fe sum feature, key: brand_level4, on: buzhi_1h_2h_rt
get fe mean feature, key: brand_level4, on: favorite_1h_2h_rt
get fe sum feature, key: brand_level4, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: brand_level4, on: orders_1h...
load fe sum feature from pickle file: key: brand_level4, on: orders_1h...
load fe mean feature from pickle file: key: brand_level4, on: orders_2h...
load fe sum feature from pickle file: key: brand_level4, on: orders_2h...
load fe mean feature from pickle file: key: brand_level4, on: order_1h_2h..

In [45]:
data=pd.concat([train_data,
                test_data,
               ]).reset_index(drop=True)

data=create_features(data)

get fe mean feature, key: baike_id_1h, on: comments_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: comments_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: zhi_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: zhi_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: buzhi_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: buzhi_1h_2h_rt
get fe mean feature, key: baike_id_1h, on: favorite_1h_2h_rt
get fe sum feature, key: baike_id_1h, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: baike_id_1h, on: orders_1h...
load fe sum feature from pickle file: key: baike_id_1h, on: orders_1h...
load fe mean feature from pickle file: key: baike_id_1h, on: orders_2h...
load fe sum feature from pickle file: key: baike_id_1h, on: orders_2h...
load fe mean feature from pickle file: key: baike_id_1h, on: order_1h_2h...
load fe sum feature from pickle file: key: baike_id_1h, on: order_1h_2h...
load fe mean feature from pickle file: key: baike_id_1h, on: article_id...
load fe sum 

get fe sum feature, key: url, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: url, on: orders_1h...
load fe sum feature from pickle file: key: url, on: orders_1h...
load fe mean feature from pickle file: key: url, on: orders_2h...
load fe sum feature from pickle file: key: url, on: orders_2h...
load fe mean feature from pickle file: key: url, on: order_1h_2h...
load fe sum feature from pickle file: key: url, on: order_1h_2h...
load fe mean feature from pickle file: key: url, on: article_id...
load fe sum feature from pickle file: key: url, on: article_id...
get fe mean feature, key: dayofweek, on: comments_1h_2h_rt
get fe sum feature, key: dayofweek, on: comments_1h_2h_rt
get fe mean feature, key: dayofweek, on: zhi_1h_2h_rt
get fe sum feature, key: dayofweek, on: zhi_1h_2h_rt
get fe mean feature, key: dayofweek, on: buzhi_1h_2h_rt
get fe sum feature, key: dayofweek, on: buzhi_1h_2h_rt
get fe mean feature, key: dayofweek, on: favorite_1h_2h_rt
get fe sum feature, key:

get fe mean feature, key: mall_brand, on: zhi_1h_2h_rt
get fe sum feature, key: mall_brand, on: zhi_1h_2h_rt
get fe mean feature, key: mall_brand, on: buzhi_1h_2h_rt
get fe sum feature, key: mall_brand, on: buzhi_1h_2h_rt
get fe mean feature, key: mall_brand, on: favorite_1h_2h_rt
get fe sum feature, key: mall_brand, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: mall_brand, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand, on: article_id...
load fe sum feature from pickle file: key: mall_brand, on: article_id...
get fe mean feature, key: mall_level1, on: comments_1h_2h_rt
get fe sum f

load fe mean feature from pickle file: key: brand_level3, on: article_id...
load fe sum feature from pickle file: key: brand_level3, on: article_id...
get fe mean feature, key: brand_level4, on: comments_1h_2h_rt
get fe sum feature, key: brand_level4, on: comments_1h_2h_rt
get fe mean feature, key: brand_level4, on: zhi_1h_2h_rt
get fe sum feature, key: brand_level4, on: zhi_1h_2h_rt
get fe mean feature, key: brand_level4, on: buzhi_1h_2h_rt
get fe sum feature, key: brand_level4, on: buzhi_1h_2h_rt
get fe mean feature, key: brand_level4, on: favorite_1h_2h_rt
get fe sum feature, key: brand_level4, on: favorite_1h_2h_rt
load fe mean feature from pickle file: key: brand_level4, on: orders_1h...
load fe sum feature from pickle file: key: brand_level4, on: orders_1h...
load fe mean feature from pickle file: key: brand_level4, on: orders_2h...
load fe sum feature from pickle file: key: brand_level4, on: orders_2h...
load fe mean feature from pickle file: key: brand_level4, on: order_1h_2h..